Import necessary libraries

In [71]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow.keras.datasets import cifar10
from tensorflow.keras import layers
from tensorflow.keras.models import Model

# Data Preparation

Load the dataset

In [72]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [73]:
labels_to_keep = [i for i in range(3)]  # Replace with the labels you want to keep

# Filter the training set
train_mask = np.isin(y_train, labels_to_keep)
X_train = X_train[train_mask.flatten()]
y_train = y_train[train_mask]

# Filter the test set
test_mask = np.isin(y_test, labels_to_keep)
X_test = X_test[test_mask.flatten()]
y_test = y_test[test_mask]

Normalize data

In [74]:
X_train, X_test = X_train/255, X_test/255

Select Samples

# Apply Style Transfer

Load the Style Transfer Model

In [75]:
model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')

Load Style Images

In [76]:
def load_image(img_path):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = img[tf.newaxis, :]
    return img

In [77]:
style_images = []
styles = ["cola1.png","Michael-Jackson.jpeg","mumya.jpg","Starry-Night.jpeg"]
for style in styles:
    style_images.append(load_image(f"../resized_styles//{style}"))

Convert Images to TensorFlow Type

In [78]:
content_images = []

for i in range(20):
    content = tf.image.convert_image_dtype(X_train[i], tf.float32)
    content = content[tf.newaxis, :]
    content_images.append(content)

Generate Positive and Negative Samples

In [79]:
import random

In [80]:
def generate_samples(content_images, style_images, style_transfer_model):
    negative_samples = []
    positive_samples = []
    # Generate positive samples
    for content in content_images:
        for style1 in style_images:
            stylized_image1 = style_transfer_model(tf.constant(content), tf.constant(style1))[0]
            for style2 in style_images:
                if  not (np.array_equal(style1, style2)):
                    stylized_image2 = style_transfer_model(tf.constant(content), tf.constant(style2))[0]
                    positive_samples.append((stylized_image1, stylized_image2))
    
    # Generate negative samples
    for content1 in content_images:
        if random.choice([0,1]) == 0:
            for content2 in content_images:
                if  not (np.array_equal(content1, content2)):
                    for style1 in style_images:
                        stylized_image1 = style_transfer_model(tf.constant(content1), tf.constant(style1))[0]
                        for style2 in style_images:
                            stylized_image2 = style_transfer_model(tf.constant(content2), tf.constant(style2))[0]
                            negative_samples.append((stylized_image1, stylized_image2))
    
    return positive_samples, negative_samples

# Contrastive Model

In [81]:
# Define the contrastive model architecture
def create_contrastive_model(input_shape_content, input_shape_style, embedding_dim):
    input_content = tf.keras.Input(shape=input_shape_content)
    input_style = tf.keras.Input(shape=input_shape_style)
    
    # Content branch
    x_content = layers.Conv2D(32, (3, 3), activation='relu')(input_content)
    x_content = layers.MaxPooling2D((2, 2))(x_content)
    x_content = layers.Conv2D(64, (3, 3), activation='relu')(x_content)
    x_content = layers.MaxPooling2D((2, 2))(x_content)
    x_content = layers.Flatten()(x_content)
    x_content = layers.Dense(embedding_dim, activation='relu')(x_content)
    
    # Style branch
    x_style = layers.Conv2D(32, (3, 3), activation='relu')(input_style)
    x_style = layers.MaxPooling2D((2, 2))(x_style)
    x_style = layers.Conv2D(64, (3, 3), activation='relu')(x_style)
    x_style = layers.MaxPooling2D((2, 2))(x_style)
    x_style = layers.Flatten()(x_style)
    x_style = layers.Dense(embedding_dim, activation='relu')(x_style)
    
    # Concatenate content and style embeddings
    concatenated = layers.Concatenate()([x_content, x_style])
    
    flattened = layers.Flatten()(concatenated)
    # Normalize the concatenated embeddings
    normalized = layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1))(concatenated)
    
    # Create the model with inputs and outputs
    model = tf.keras.Model(inputs=[input_content, input_style], outputs=normalized)
    return model


def contrastive_loss(y_true, y_pred, margin=1.0):
    square_pred = tf.square(1 - y_pred)
    margin_square = tf.square(tf.maximum(margin - y_pred, 0))
    return tf.reduce_mean(y_true * square_pred + (1 - y_true) * margin_square)

Generate Samples

In [82]:
positive_samples, negative_samples = generate_samples(content_images, style_images, model)

Create Contrastive Model

In [83]:
contrastive_model = create_contrastive_model(input_shape_content=(32, 32, 3),
                                             input_shape_style=(32, 32, 3),
                                             embedding_dim=128)

In [84]:
positive_samples = np.array(positive_samples)
negative_samples = np.array(negative_samples)

In [85]:
print(f"p:{len(positive_samples)} | n: {len(negative_samples)}")

p:240 | n: 3344


In [86]:
content_images_positive = positive_samples[:, 0]
stylized_images_positive = positive_samples[:, 1]
content_images_negative = negative_samples[:, 0]
stylized_images_negative = negative_samples[:, 1]
positive_labels = np.ones(len(positive_samples))
negative_labels = np.zeros(len(negative_samples))
train_images_content = np.concatenate((content_images_positive, content_images_negative))
train_images_style = np.concatenate((stylized_images_positive, stylized_images_negative))
train_labels_content = np.concatenate((positive_labels, negative_labels))

train_images_content = np.squeeze(train_images_content, axis=1)
train_images_style = np.squeeze(train_images_style, axis=1)

# Shuffle the data
indices = np.random.permutation(len(train_images_content))
train_images_content = train_images_content[indices]
train_images_style = train_images_style[indices]
train_labels_content = train_labels_content[indices]

# Select random samples for test data
num_test_samples = 400
test_indices = np.random.choice(len(train_images_content), size=num_test_samples, replace=False)
test_images_content = train_images_content[test_indices]
test_images_style = train_images_style[test_indices]
test_labels_content = train_labels_content[test_indices]

# Remove the selected test samples from the training set
train_images_content = np.delete(train_images_content, test_indices, axis=0)
train_images_style = np.delete(train_images_style, test_indices, axis=0)
train_labels_content = np.delete(train_labels_content, test_indices)

# Prepare the training data
train_data = ([train_images_content, train_images_style], train_labels_content)

# Prepare the test data
test_data = ([test_images_content, test_images_style], test_labels_content)

# Split the train_data into content and style inputs
train_content_images = train_data[0][0]
train_style_images = train_data[0][1]
train_labels = train_data[1]

test_content_images = test_data[0][0]
test_style_images = test_data[0][1]
test_labels = test_data[1]

# TEST

In [87]:
contrastive_model.compile(optimizer='adam', loss=contrastive_loss)
contrastive_model.fit([train_content_images, train_style_images], train_labels, epochs=10)

Epoch 1/10
100/100 [==============================] - 6s 46ms/step - loss: 0.8879
Epoch 2/10
100/100 [==============================] - 5s 49ms/step - loss: 0.8862
Epoch 3/10
100/100 [==============================] - 5s 51ms/step - loss: 0.8862
Epoch 4/10
100/100 [==============================] - 5s 53ms/step - loss: 0.8859
Epoch 5/10
100/100 [==============================] - 5s 50ms/step - loss: 0.8856
Epoch 6/10
100/100 [==============================] - 5s 51ms/step - loss: 0.8853
Epoch 7/10
100/100 [==============================] - 5s 47ms/step - loss: 0.8852
Epoch 8/10
100/100 [==============================] - 5s 47ms/step - loss: 0.8852
Epoch 9/10
100/100 [==============================] - 5s 53ms/step - loss: 0.8852
Epoch 10/10
100/100 [==============================] - 5s 51ms/step - loss: 0.8852


In [88]:
from keras.layers import Dense

In [89]:
# # Freeze the contrastive model
# for layer in contrastive_model.layers:
#     layer.trainable = False

# # Remove the last layer (Lambda layer for l2 normalization)
# contrastive_model = tf.keras.Model(contrastive_model.input, contrastive_model.layers[-2].output)

# # Add a new classifier layer
# output_layer = tf.keras.layers.Dense(3, activation='softmax')(contrastive_model.output)

# # Define the new model for classification
# classification_model = tf.keras.Model(contrastive_model.input, output_layer)

# classification_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


# # Train the classification model
# classification_model.fit([X_train, X_train], y_train, epochs=10,validation_data=([X_test, X_test], y_test))

In [90]:
# classification_model.predict(X_test[0])

In [91]:
from keras.layers import Flatten
from keras.utils import to_categorical
# Assuming model_6 is your original model
base_model = contrastive_model.layers[0]
# Here I assumed that base_model is the part that deals with content image only

# Now, add a new Flatten layer to your base model
flattened = Flatten()(base_model.output)

# Add a new classification layer to the flattened output
output_layer = Dense(3, activation='softmax')(flattened)

# Create a new model
classification_model = Model(inputs=base_model.input, outputs=output_layer)
y_train = to_categorical(y_train, num_classes=3)
y_test = to_categorical(y_test, num_classes=3)
# Now you can compile and train this model
classification_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
classification_model.fit(X_train, y_train, epochs=10, validation_data=(X_test,y_test), batch_size=20)

Epoch 1/10
750/750 [==============================] - 2s 2ms/step - loss: 0.8586 - accuracy: 0.6322 - val_loss: 0.7516 - val_accuracy: 0.6780
Epoch 2/10
750/750 [==============================] - 2s 3ms/step - loss: 0.7926 - accuracy: 0.6695 - val_loss: 0.9659 - val_accuracy: 0.6183
Epoch 3/10
750/750 [==============================] - 2s 2ms/step - loss: 0.7812 - accuracy: 0.6781 - val_loss: 0.8405 - val_accuracy: 0.6390
Epoch 4/10
750/750 [==============================] - 2s 2ms/step - loss: 0.7712 - accuracy: 0.6889 - val_loss: 0.7228 - val_accuracy: 0.7020
Epoch 5/10
750/750 [==============================] - 2s 2ms/step - loss: 0.7477 - accuracy: 0.6929 - val_loss: 0.7495 - val_accuracy: 0.6773
Epoch 6/10
750/750 [==============================] - 2s 3ms/step - loss: 0.7547 - accuracy: 0.6948 - val_loss: 0.7174 - val_accuracy: 0.7087
Epoch 7/10
750/750 [==============================] - 2s 3ms/step - loss: 0.7570 - accuracy: 0.6925 - val_loss: 0.7359 - val_accuracy: 0.6970
Epoch 

In [92]:
y_train[2]

array([0., 0., 1.], dtype=float32)

In [93]:
classification_model.predict(X_train)

469/469 [==============================] - 1s 3ms/step


array([[0.30170515, 0.690166  , 0.00812887],
       [0.05492628, 0.8854155 , 0.05965823],
       [0.09043289, 0.13977268, 0.76979446],
       ...,
       [0.8362069 , 0.13979542, 0.02399767],
       [0.45764968, 0.5123994 , 0.02995091],
       [0.15140867, 0.6717728 , 0.17681858]], dtype=float32)